# Correcting sextractor test catalogs

We need to construct big images in order to launch sextractor over it and use PSFEx as a benchmark tool.

When doing so, the images extracted by sextractor are transposed and is hard to reverse as we should dig into Sextractor code. The easy way to solve this is to transpose the star images used for testing the model.

This script is about that.

In [50]:
import numpy as np
import scipy.stats as sst
import matplotlib.pyplot as plt
import progressbar as pb
import os
import glob
import shutil

from astropy.io import fits
from shapepipe.pipeline import file_io

In [51]:
def create_fits(fits_path, data_dict, fits_ex):
    """
    Create a .fits file
    INPUTS:
        fits_path: path to the future .fits file
        data_dict: dictionnary where data is stored (usually {'VIGNET': ..., 'XWIN_IMAGE': ..., ...})
        fits_ex: path to an existing .fits file used as a model
    """
    fits_file = file_io.FITSCatalog(fits_path, open_mode = file_io.BaseCatalog.OpenMode.ReadWrite, SEx_catalog=True)
    fits_file.save_as_fits(data_dict, sex_cat_path = basic_fits)
    

In [52]:
# Location of the file containing raw simulated stars
CCD_number = 38
old_data_path = '/Users/tliaudat/Documents/PhD/codes/venv_p3/JB-data/CCD-38_sexdataset/'

# Directory used to store generated catalogs
# output_path = '/Users/mschmitz/jbonnin/Data/CFIS_PSF/CCD-{}_newsexdataset'.format(CCD_number)
new_data_path = '/Users/tliaudat/Documents/PhD/codes/venv_p3/JB-data/TL_sexcatalog/CCD-{}_newsexdataset/'.format(CCD_number)

# Path of a model file for creating .fits catalogs
basic_fits = '/Users/tliaudat/Documents/PhD/codes/venv_p3/JB-data/other-JB-data/all_w3_star_cat/star_selection-2079613-0.fits'

dataset_pattern = 'test-star_selection-*-38.fits'


In [53]:
# Load the data
old_dataset_paths = glob.glob(old_data_path + dataset_pattern)

old_dataset_paths.sort()

k = 0 # Catalog number
print('\nOld Dataset path [%d]: \n'%(k) + old_dataset_paths[k])



Old Dataset path [0]: 
/Users/tliaudat/Documents/PhD/codes/venv_p3/JB-data/CCD-38_sexdataset/test-star_selection-0000000-38.fits


In [54]:
# Extract the names of the files
data_file_names = []

for old_path in old_dataset_paths:
    data_name = old_path.split('/',20)[-1]
    #data_file_names.append(data_name)
    
    model = fits.open(old_path)
    stars = model[2].data['VIGNET']
    for it in range(stars.shape[0]):
        stars[it,:,:] = stars[it,:,:].T
    
    test_dict = {'VIGNET': stars, 'XWIN_IMAGE': model[2].data['XWIN_IMAGE'], 
                 'YWIN_IMAGE': model[2].data['YWIN_IMAGE'], 'NUMBER': model[2].data['NUMBER']}

    fits_path = new_data_path + data_name
    create_fits(fits_path, test_dict, basic_fits)
    
    
# old_dataset_paths[0].split('/', 20)[-1]